# Example: Image Reconstruction
We know turn our attention to a complicated, "large scale" demonstration of the techniques which we have just reviewed in the previous notebooks. Here, we will look at a compressed sensing *image reconstruction* problem.

We will assume that we have some manner by which we can acquire random samples of a square image which is of size ($512\times 512$), which comes out to $N = 262144$, a problem size which far surpasses our ability to use random, dense sampling matrices.

We need to devise both a sampling method and a reconstruction approach which are well suited for problems of this size.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pywt as pw
import scipy.fftpack as fp
import scipy.linalg as la       
import scipy.signal as sig
from scipy import misc

In [ ]:
def MSE(x,y):
    mseval = np.mean(np.power(x-y,2))    
    return mseval

def normalize(x):
    y = (x - min(x))/(max(x)-min(x))
    return y

class SRM:
    def __init__(self,m,n,blksize):
        self.n = n
        self.m = m
        self.blksize = blksize
        self.random_perm = np.random.permutation(n)
        self.select_vect = np.random.permutation(n)[:m]
        self.D = fp.dct(np.eye(blksize),norm='ortho',type=3)
        self.blkshape = (blksize,n/blksize)
        
    def forward(self,x):
        y = np.asarray(x)
        y = y[self.random_perm]
        y = np.reshape(y,self.blkshape)
        y = np.dot(self.D,y)
        y = np.reshape(y,(self.n,1))
        y = y[self.select_vect]
        return y
    
    def transpose(self,y):
        x = np.zeros((self.n,1))
        x[self.select_vect] = y
        x = np.reshape(x,self.blkshape)
        x = np.dot(self.D.T,x);
        x = np.reshape(x,(self.n,1))
        x = x[np.argsort(self.random_perm)]
        
        return x

    
def DoubleExpand(x):
    r,c = np.shape(x)
    X = np.zeros((2*r,2*c))
    X[0::2,0::2] = x
    X[1::2,0::2] = x
    X[0::2,1::2] = x
    X[1::2,1::2] = x
    
    return X
    
    
def bishrink(w,wParent,beta):
    s = np.power(np.abs(w),2)
    sParent = np.power(np.abs(wParent),2)
    R = np.sqrt(s + sParent)
    R[R<beta] = 0.0
    R = np.divide(R,R+beta)
    wFinal = np.multiply(w,R)
    
    return wFinal
    
def BivariateShrinkage(W,end_level,lam):
    windowsize = 3
    eps = 1e-6
    windowfilt = np.ones((windowsize,windowsize))/(windowsize*windowsize)
    L = np.shape(W)[0]
    # Check noise sig on the highest detail diagonal
    NoiseSig = np.median(abs(W[L-1][2]))/0.6745
    NoiseVar = NoiseSig*NoiseSig
    
    for scale in (range(0,L)[:end_level:-1]):
        for dir in [0,1,2]:
            w = W[scale][dir]
            wParent = DoubleExpand(W[scale-1][dir])
            
            Wsig = sig.convolve2d(np.power(w,2),windowfilt,mode='same',boundary='symm')
            Ssig = np.sqrt(np.max(Wsig - NoiseVar,eps))
            
            T = np.sqrt(3)*NoiseVar/Ssig
            
            W[scale][dir] = bishrink(w,wParent,T*lam)
    
    return W
    
def ISTImageRecovery(F,y,imshape,lam,iters,xtrue):
    wfilter = 'db1'
    vecshape = (F.n,1)
    L = 5
    x = np.zeros((F.n,1))
    xlast = np.zeros((F.n,1))
    for i in range(1,iters):
        r = y - F.forward(x)
        z = x + F.transpose(r)
        # Transform into wavelet coefficients
        zim = np.reshape(z,imshape)
        wim = pw.wavedec2(zim,wfilter,mode='sym',level=L)
        # From Tuple->List
        wim = list(wim)
        for l in range(0,L+1):
            wim[l] = list(wim[l])
        # Apply Shrinkage
        wim = BivariateShrinkage(wim,1,lam)
        # Back to image space  
        xim = pw.waverec2(wim,wfilter,mode='sym')
        xlast = x
        x = np.reshape(xim,vecshape)
        
        conv = MSE(x,xlast)
        mseval = MSE(x,xtrue)
        
        if conv < 1e-6:
            break
        
        print "[%d] Conv : %.3e | MSE : %.3e" % (i,conv,mseval)
        
    x = np.reshape(x,imshape)
    return x

In [ ]:
im = misc.lena().astype(np.float32)

N = np.size(im)
M = int(round(0.25*N))
imshape = np.shape(im)

F = SRM(M,N,32)
x = np.reshape(im,(N,1))
x = normalize(x)
y = F.forward(x)

xt = ISTImageRecovery(F,y,imshape,20,100,x)

In [ ]:
xim = np.reshape(x,imshape)
plt.matshow(np.reshape(x,imshape));
plt.gray()

plt.matshow(xt);
plt.gray()

plt.matshow(abs(xim-xt));
plt.jet()